In [1]:
# Load libraries
from pandas_plink import read_plink1_bin
import pandas as pd
import transprs as tprs

# Read the population file
population = read_plink1_bin("./data/EUR.bed",
                             "./data/EUR.bim",
                             "./data/EUR.fam")
# Read sumstats file
sumstats = pd.read_table("./data/Height.gwas.txt.gz")

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  2.42it/s]


# Xử lý processor 1

Chạy tương tự như bên single population

In [2]:
# Create the DataProcessor object
processor = tprs.datasets.DataProcessor(sumstats=sumstats, population=population)

In [3]:
phenotype = pd.read_table("./data/EUR.height")
processor.add_phenotype(phenotype)

Phenotype stored in .phenotype


/scratch/user/uqdpham4/.conda/envs/transprs/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [4]:
# Run preprocessing
processor.clean_snps()
processor.filter_imputed(info=0.9)
processor.extract_intersection()
processor.check_beta_se()
processor.flip_reverse()
processor.compute_pca(n_components=6)
processor.split_chromosomes()
processor.sort_snps_chr()
processor.cross_validation_split(id_col="FID",k_folds=5,n_repeats=10)

/scratch/user/uqdpham4/VGP/transprs/datasets/classes.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.sumstats["CHR"][


PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp.log.
Options in effect:
  --bfile tmp
  --indep-pairwise 200 50 0.25
  --out tmp

128656 MB RAM detected; reserving 64328 MB for main workspace.
551892 variants loaded from .bim file.
475 people (227 males, 248 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999896.
551892 variants and 475 people pass filters and QC.
Note: No phenotypes present.
Pruned 20659 variants from chromosome 1, leaving 21506.
Pruned 21363 variants from chromosome 2, leaving 21295.
Pruned 17875 var

In [5]:
# Run the method PRS method: clumping
tprs.methods.clumping(processor)

Extracting data...
Done extract data!
Clumping is running...
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp_out.log.
Options in effect:
  --bfile tmp
  --clump tmp_ss
  --clump-field P
  --clump-kb 250
  --clump-p1 1
  --clump-r2 0.5
  --clump-snp-field SNP
  --out tmp_out

128656 MB RAM detected; reserving 64328 MB for main workspace.
Allocated 20353 MB successfully, after larger attempt(s) failed.
551892 variants loaded from .bim file.
475 people (227 males, 248 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899%

KeyboardInterrupt: 

In [5]:
# Run the method PRS method: double_weight
tprs.methods.double_weight(processor)

Double weight method is running...
[1] "Adjusted BETA is done!"
Done Double weight!
The double weight result stores in .adjusted_ss['double_weight']!
--- Done in 0:00:35 ---


In [ ]:
tprs.scoring.generate_prs(processor,use_col="OR",method="clumping")

In [6]:
tprs.scoring.generate_prs(processor,use_col="OR",method="double_weight")

Extracting adjusted sumstats from double_weight method...
Done extract data!
Generating PRS...
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp_results.log.
Options in effect:
  --bfile tmp
  --out tmp_results
  --q-score-range tmp_range_list tmp_SNP.pvalue
  --score tmp_ss 3 4 9 header

128656 MB RAM detected; reserving 64328 MB for main workspace.
551892 variants loaded from .bim file.
475 people (227 males, 248 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899%

 done.
Total genotyping rate is 0.999896.
551892 variants and 475 people pass filters and QC.
Note: No phenotypes present.
--score: 264707 valid predictors loaded.
--score: 7 ranges processed.
Results written to tmp_results.*.profile.
PRS is generated!
The PRS result stores in .prs_results['double_weight']!
--- Done in 0:00:11 ---


In [ ]:
tprs.scoring.generate_prs(processor,use_col="OR",method="PRScs")

In [ ]:
tprs.metrics.r2_score_evaluation(processor,method="clumping", trait_col="Height", prs_col="SCORE")

In [7]:
tprs.metrics.r2_score_evaluation(processor,method="double_weight", trait_col="Height", prs_col="SCORE")

The best fit p-value is 0.3
The best fit result is stored in processor.prs_results['double_weight']['best_fit']
The best fit result is stored in processor.performance['double_weight']['r2_score']


In [ ]:
tprs.metrics.r2_score_evaluation(processor,method="PRScs", trait_col="Height", prs_col="SCORE")

# Xử lý processor 2

Ví dụ nên để dup cái processor ra

In [8]:
processor2 = processor

# Combine multiple population

Có thể kết hợp bất kỳ methods nào từ 2 hay nhiều processor. Hiện tại chỉ hỗ trợ combine best fit PRS nên bắt buộc phải chạy evaluation cho từng processor trước khi combine. Sau khi combine sẽ sinh ra 1 thằng `adjusted_ss['multipop']` cho mỗi processor nên cần chạy tiếp PRS và evaluation.

In [11]:
from transprs.combine.multipop import combine_multipop
combine_multipop([processor,processor2], methods=["double_weight","double_weight"],trait_col="Height", prs_col="SCORE",
                key_ss="multipop", use_col="OR")

Estimating mixing weights...
Mixing weights are: [0.14153556 0.        ]
Estimating mixing is done!
Adjusted BETA...
Adjusted BETA is done
The clumping result stores in .adjusted_ss['multipop']!
--- Done in 0:00:01 ---


# PRScsx

PRScsx cần thêm LD reference. Cần chuẩn bị folder đó như trong hướng dẫn: https://github.com/getian107/PRScsx

PRScsx cũng tương tự, mỗi sumstats từ population khác nhau sẽ có processor tương ứng sẽ 1 thằng `adjusted_ss['PRScsx']` cho mỗi processor nên cần chạy tiếp PRS và evaluation.

In [21]:
# Run the method PRS method: PRScs
from transprs.methods.multipop import prscsx
prscsx([processor,processor2],populations=["EUR","EAS"],ldref_dir="./data/ld_ref/", use_col="OR",N=[5000,5000])

Extracting data...
Done extract data!
PRScsx is running...
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp1.log.
Options in effect:
  --bim tmp.bim
  --chr 1
  --make-just-bim
  --out tmp1

128656 MB RAM detected; reserving 64328 MB for main workspace.
Allocated 20353 MB successfully, after larger attempt(s) failed.
42165 out of 551892 variants loaded from .bim file.
Using 1 thread (no multithreaded calculations invoked).
42165 variants pass filters and QC.
--make-just-bim to tmp1.bim ... done.
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp2.log.
Options in effect:
  --bim tmp.bim
  --chr 2
  --make-just-bim
  --out tmp2

128656 MB RAM detected; reserving 64328 MB for main workspace.
Allocated 20353 MB successfully, after larger attempt(s)

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp17.log.
Options in effect:
  --bim tmp.bim
  --chr 17
  --make-just-bim
  --out tmp17

128656 MB RAM detected; reserving 64328 MB for main workspace.
Allocated 20353 MB successfully, after larger attempt(s) failed.
17122 out of 551892 variants loaded from .bim file.
Using 1 thread (no multithreaded calculations invoked).
17122 variants pass filters and QC.
--make-just-bim to tmp17.bim ... done.
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp18.log.
Options in effect:
  --bim tmp.bim
  --chr 18
  --make-just-bim
  --out tmp18

128656 MB RAM detected; reserving 64328 MB for main workspace.
Allocated 20353 MB successfully, after larger attempt(s) failed.
16059 out of 551892 variants loaded from .b

mkdir: cannot create directory ‘tmp’: File exists


Applying PRScsx for CHR 1...


--ref_dir=./data/ld_ref/
--bim_prefix=./tmp1
--sst_file=['./tmp0_ss', './tmp1_ss']
--a=1
--b=0.5
--phi=0.01
--n_gwas=[5000, 5000]
--pop=['EUR', 'EAS']
--n_iter=1000
--n_burnin=500
--thin=5
--out_dir=./tmp
--out_name=tmp
--chrom=['1']
--meta=FALSE
--seed=None


##### process chromosome 1 #####
... parse reference file: ./data/ld_ref//snpinfo_mult_ukbb_hm3 ...
... 108029 SNPs on chromosome 1 read from ./data/ld_ref//snpinfo_mult_ukbb_hm3 ...
... parse bim file: ./tmp1.bim ...
... 42165 SNPs on chromosome 1 read from ./tmp1.bim ...
... parse EUR sumstats file: ./tmp0_ss ...
... 264707 SNPs read from ./tmp0_ss ...
... 5537 common SNPs in the EUR reference, EUR sumstats, and validation set ...
... parse EAS sumstats file: ./tmp1_ss ...
... 264707 SNPs read from ./tmp1_ss ...


Traceback (most recent call last):
  File "/scratch/user/uqdpham4/VGP/transprs/methods/multipop/prscsx/PRScsx.py", line 293, in <module>
    main()
  File "/scratch/user/uqdpham4/VGP/transprs/methods/multipop/prscsx/PRScsx.py", line 244, in main
    sst_dict[pp] = parse_genet.parse_sumstats(
  File "/scratch/user/uqdpham4/VGP/transprs/methods/multipop/prscsx/parse_genet.py", line 121, in parse_sumstats
    set(zip(sst_dict["SNP"], sst_dict["A1"], sst_dict["A2"]))
KeyboardInterrupt


KeyboardInterrupt: 